In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
directory_path = '/content/drive/MyDrive/info 443'

cos_df_path = os.path.join(directory_path, 'cos_df.csv')
anime_sentiment_path = os.path.join(directory_path, 'anime_with_sentiment.csv')

cos_df = pd.read_csv(cos_df_path, index_col=0)
df = pd.read_csv(anime_sentiment_path, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/info 443/cos_df.csv'

In [ ]:
display(cos_df.head())

0         1         2         3         4         5         6  \
0  1.000000  0.516253  0.428031  0.563448  0.295962  0.387724  0.626643   
1  0.516253  1.000000  0.445281  0.413750  0.315341  0.374002  0.446435   
2  0.428031  0.445281  1.000000  0.358100  0.326103  0.406346  0.355004   
3  0.563448  0.413750  0.358100  1.000000  0.244203  0.357018  0.525956   
4  0.295962  0.315341  0.326103  0.244203  1.000000  0.344179  0.257963   

          7         8         9  ...       190       191       192       193  \
0  0.584211  0.562289  0.464091  ...  0.273201  0.534407  0.270049  0.521090   
1  0.418691  0.472154  0.460869  ...  0.288456  0.346611  0.278577  0.315027   
2  0.334386  0.401617  0.487899  ...  0.271213  0.251592  0.253693  0.192785   
3  0.495607  0.462006  0.375026  ...  0.221214  0.461192  0.231961  0.456855   
4  0.260913  0.306183  0.380169  ...  0.189777  0.164820  0.177325  0.124980   

        194       195       196       197       198       199  
0  0.510990  0.460841  0.569340  0.093505  0.467039  0.362994  
1  0.286668  0.447115  0.386337  0.097718  0.366070  0.345400  
2  0.164052  0.386511  0.257446  0.118008  0.303822  0.312025  
3  0.433806  0.381683  0.479191  0.091720  0.389593  0.305188  
4  0.130919  0.282799  0.188279  0.073040  0.204457  0.214156  

[5 rows x 200 columns]

In [ ]:
display(df.head())

title                           genre_clean  \
0                     Cowboy Bebop         action,adventure,drama,sci fi   
1  Cowboy Bebop: Tengoku no Tobira           action,drama,mystery,sci fi   
2                           TRIGUN  action,adventure,comedy,drama,sci fi   
3               Witch Hunter ROBIN     action,drama,mystery,supernatural   
4                   Bouken Ou Beet        adventure,fantasy,supernatural   

                                      clean_synopsis  score  episodes  \
0  enter world distant future bounty hunter roam ...     86        26   
1  cowboy bebop crew travel star learn largest bo...     82         1   
2  vash stampede wanted man habit turning entire ...     80        26   
3  robin sena powerful craft user drafted stnj gr...     68        26   
4  dark century people suffering rule devil vande...     65        52   

   sentiment  
0    0.93290  
1    0.29210  
2    0.04655  
3    0.65910  
4    0.02110

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
numerical_cols = ['score', 'episodes', 'sentiment']

scaler = MinMaxScaler()

scaled_numerical_data = scaler.fit_transform(df[numerical_cols])

numerical_features_scaled_df = pd.DataFrame(scaled_numerical_data, columns=numerical_cols)

NameError: name 'df' is not defined

In [ ]:
tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2', use_idf=True, smooth_idf=True)
tv_matrix = tv.fit_transform(df['clean_synopsis'])
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names_out()
tv_df = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

NameError: name 'df' is not defined

In [ ]:
df2 = pd.concat([tv_df, numerical_features_scaled_df], axis=1)

NameError: name 'tv_df' is not defined

In [ ]:
cosine_sim_matrix = cosine_similarity(df2)
cosine_df2 = pd.DataFrame(cosine_sim_matrix)

NameError: name 'df2' is not defined

In [ ]:
def recommendations(title):

    lower_title = title.lower()

    if lower_title not in df['title'].str.lower().values:
        return f"'{title}' not found in the dataset. Please provide a title from the dataset."

    idx = df[df['title'].str.lower() == lower_title].index[0]
    sim_scores = list(enumerate(cosine_df2.iloc[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # Get top 3, excluding itself
    item_indices = [i[0] for i in sim_scores]
    scores = [i[1] * 100 for i in sim_scores] # Convert to percentage

    recommendations_df = pd.DataFrame({
        'Rank': range(1, len(item_indices) + 1), # Add rank column starting at 1
        'Recommended Title': df['title'].iloc[item_indices].tolist(),
        'Percentage Score': [f"{s:.2f}%" for s in scores] # Format as percentage string
    })

    return recommendations_df

In [ ]:
recommendations('trigUN')

Recommended Title  Cosine Score
0           MONSTER      0.487899
1     HUNTER×HUNTER      0.473676
2     Mononoke-hime      0.471290

In [ ]:
recommendations('Naruto')

Recommended Title  Cosine Score
0                             Eyeshield 21      0.567839
1                      Tennis no Ouji-sama      0.543054
2  Rurouni Kenshin: Meiji Kenkaku Romantan      0.501948

In [ ]:
recommendations('EYEshield 21')

Recommended Title  Cosine Score
0               NARUTO      0.567839
1  Tennis no Ouji-sama      0.501828
2              MONSTER      0.481393

In [ ]:
while True:
    anime_title = input("Enter an anime title: ")
    recommended_anime = recommendations(anime_title)

    if isinstance(recommended_anime, str):
        print(recommended_anime)
    else:
        display(recommended_anime)
        break

Enter an anime title: Naruto


Recommended Title  Cosine Score
0                             Eyeshield 21      0.567839
1                      Tennis no Ouji-sama      0.543054
2  Rurouni Kenshin: Meiji Kenkaku Romantan      0.501948

In [ ]:
recommendations('Naruto')

NameError: name 'df' is not defined